# w3Quiz2. Multiply hypotheses tests

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sts
import statsmodels.stats.multitest as msts

import matplotlib.pyplot as plt

___
Выберите задачи, в которых обязательно нужно применять поправку на множественную проверку гипотез
- [X] Выбор инвестиционных фондов с помощью сравнения доходности каждого из них с доходностью базового актива по нескольким историческим периодам
- [X] Сравнение эффективности лечения пациентов в подгруппах по большому количеству признаков
- [X] Попарное сравнение средних большого количества выборок
- [X] Локализация различий в активности мозга испытуемых в разных экспериментальных условиях

___
Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора.

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [4]:
df = pd.read_csv('data/AUCs.txt', sep='\t', index_col=0)
df.head()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
adult (sample),0.763,0.768,0.771,0.798
breast cancer,0.599,0.591,0.590,0.569
breast cancer wisconsin,0.954,0.971,0.968,0.967
cmc,0.628,0.661,0.654,0.657
ionosphere,0.882,0.888,0.886,0.898


In [17]:
p_data = []
for i, lhp in enumerate(df.columns):
    for j, rhp in enumerate(df.columns):
        if i >= j:
            continue
        p_data.append([
            lhp,
            rhp,
            sts.wilcoxon(df[lhp].values, df[rhp].values).pvalue
        ])

In [19]:
comp_data = pd.DataFrame.from_records(p_data)
comp_data.columns = ['clfr1', 'clfr2', 'p-value']
comp_data.sort_values(by='p-value')

,clfr1,clfr2,p-value
0,C4.5,C4.5+m,0.010757
2,C4.5,C4.5+m+cf,0.015906
5,C4.5+cf,C4.5+m+cf,0.022909
3,C4.5+m,C4.5+cf,0.046333
4,C4.5+m,C4.5+m+cf,0.327826
1,C4.5,C4.5+cf,0.861262


- [X] C4.5
- [X] C4.5+m

Сколько статистически значимых на уровне 0.05 различий мы обнаружили?
- 4

___
Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?
- m

___
Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [22]:
msts.multipletests(comp_data['p-value'], method='holm')

(array([False, False, False, False, False, False]),
 array([0.0645428 , 0.86126233, 0.07953222, 0.13899819, 0.65565135,
        0.0916364 ]),
 0.008512444610847103,
 0.008333333333333333)

- 0

___
Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [23]:
msts.multipletests(comp_data['p-value'], method='fdr_bh')

(array([ True, False,  True, False, False,  True]),
 array([0.0458182 , 0.86126233, 0.0458182 , 0.06949909, 0.39339081,
        0.0458182 ]),
 0.008512444610847103,
 0.008333333333333333)

- 3

___
Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?
- Всегда корректно, это же метод Бенджамини-Хохберга
- [X] Некорректно: статистики, соответствующие разным гипотезам, зависимы, так что предположения метода Бенджамини-Хохберга не выполняются
- Корректно: статистики, соответствующие разным гипотезам, независимы, так что предположения метода Бенджамини-Хохберга выполняются

___
Мы подозреваем, что в проведённом с C4.5 эксперименте на самом деле классификаторы сильнее отличаются друг от друга, просто нам не удалось это заметить. Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?

- [ ] Закрыть глаза на эффект множественной проверки гипотез и не делать никакой поправки
- [X] Взять больше датасетов
- [ ] Попробовать настроить больше гиперпараметров